In [9]:
#%env CUDA_VISIBLE_DEVICES=1

import os
import re
import sys
import json
sys.path.append('./modules/multi_summ')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from collections import defaultdict

# 1. Scouter

In [2]:
%%time
from modules.scouter_handler import ScouterHandler
from modules.scouter.config import default_addr, default_newspaper_idx, default_timeline_idx
from modules.topic_generator import TopicGenerator
from modules.scouter import connectES as es

from modules.multi_doc_summary import MultiDocSummary
from modules.timeline_summary import TimelineSummary
from modules.image_selector import ImageSelectionModule
from modules.outlook import ForecastSentence

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 



CPU times: user 4.93 s, sys: 1.48 s, total: 6.41 s
Wall time: 6.08 s


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
%%time
scouter = ScouterHandler(addr=default_addr, size=1000)
multi_doc_summarizer = MultiDocSummary("", "")
imageSelector = ImageSelectionModule("")
timeline_summ = TimelineSummary("", "")
forecast = ForecastSentence("", "")

[2019-11-28 07:38:36,883 WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



[2019-11-28 07:38:38,281 WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



weight_best loaded



[2019-11-28 07:38:38,490 WARNING] From /workspace/wise_reporter/modules/image_selector.py:14: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

[2019-11-28 07:38:38,492 INFO] Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[2019-11-28 07:38:41,854 INFO] Saver not created because there are no variables in the graph to restore


[2019-11-28 07:38:42,108 WARNING] From /workspace/wise_reporter/modules/image_selector.py:17: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



graph defined


re.compile(r'[\|/|:|\*|?|"|<|>|\|]', re.UNICODE)

In [8]:
%%time
date_list = ['2019-01', '2019-02', '2019-03', '2019-04', '2019-05']

main_json = defaultdict(list)
for date in date_list:
    print(date)
    monthly_cluster = es.keywordGetter(date+"-01", index='monthly_topic')

    cluster_info = []
    content_list = []
    for c_idx, cluster in enumerate(monthly_cluster['clusters']):
        contents = []
        keywords = cluster['keywords']
        if c_idx == 0:
            doc = cluster['news_source'][0]
            
            content_list = [doc['extContent'] for doc in cluster['news_source']]
            multi_summ = multi_doc_summarizer.process_data(content_list)
            
            image_save_path = "./results/images/{}.jpg".format(date)
            image_save_path = "./results/images/{}.jpg".format(date)
            imageSelector.process_data(doc['newsTitle'], image_save_path, download_limit=50)
            
            doc_info = {'title':doc['newsTitle'],
                        'text':multi_summ,
                        'photo':image_save_path,
                        'date':doc['postingDate']}    
        else:
            doc = cluster['news_source'][0]            
                        
            doc_info = {'title':doc['newsTitle'],
                        'date':doc['postingDate']}
            
        main_json[date].append(doc_info)
        
with open("./results/main.json", "w", encoding='utf8') as fp:
    fp.write(json.dumps(main_json, ensure_ascii=False))

[2019-11-28 07:43:55,449 INFO] GET http://155.230.34.145:9200/monthly_topic/_search [status:200 request:0.016s]


2019-01


[2019-11-28 07:43:58,891 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.011s]


query already used before


[2019-11-28 07:44:03,066 INFO] GET http://155.230.34.145:9200/monthly_topic/_search [status:200 request:0.016s]


...

[2019-11-28 07:44:11,226 INFO] GET http://155.230.34.145:9200/monthly_topic/_search [status:200 request:0.017s]


2019-03


[2019-11-28 07:44:15,255 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


query already used before


IndexError: list index out of range

In [5]:
main_json.keys()

dict_keys(['2019-01', '2019-02', '2019-03', '2019-04', '2019-05'])

In [7]:
main_json['2019-01'][0]

[{'title': '구로다 일본은행 총재  인구감소 고령화가 금융기관 변혁 재촉',
  'text': '구로다 하루히코 일본은행(boj)총재는 정책금리를 현행 1.4%에서 0.5%로 동결했다. 이로써 일본 경제에 미치는 영향은 제한적일 것으로 내다봤다. 아사카와 마사쓰구 일본 재무성 재무관은 1일(현지시간) 정책결정회의를 열고 브렉시트(영국의 유럽연합(eu) 탈퇴)가 임박했다. ',
  'photo': './results/images/2019-01.jpg',
  'date': '2019-01-17'},
 {'title': '바닥 모르던 국제유가 새해 첫거래일엔 상승 마감', 'date': '2019-01-03'},
 {'title': '보우소나루 브라질 대통령  세계가 기대하는 개혁 추진할 것', 'date': '2019-01-23'},
 {'title': '미중 무역협상 갈등봉합 첫 조율 돌입', 'date': '2019-01-07'},
 {'title': '증권가 올해 수출 증가율 전망치 줄하향', 'date': '2019-01-03'}]

In [25]:
# Daily test
date = '2019-05-07'
monthly_cluster = es.keywordGetter(date, index='keyword_news')

keyword_list = []
title_list = []
content_list = []
for cluster in monthly_cluster['clusters']:
    titles = []
    contents = []
    keyword_list.append(cluster['keywords'])
    for doc in cluster['news_source']:
        titles.append(doc['newsTitle'])
        contents.append(doc['extContent'])
        
    title_list.append(titles)
    content_list.append(contents)
    
    summ_result = multi_doc_summarizer.process_data(content_list)
    
title_list

[['中외교부  美와 타협 희망, 관세로는 문제 해결 못해',
  '트럼프의  뒤집기  전술에 딜레마 빠진 中  협상 해도 문제',
  '美 관세 인상에도 침묵하는 中 속내는  버티기  준비?',
  '美 관세 인상에도 침묵하는 中 속내는  버티기  준비?',
  '최후의 무역담판  앞두고 트럼프  기세등등  시진핑  전전긍긍',
  '美 라이트하이저, 中 합의 내용서 후퇴해 관세 인상키로',
  '미중 무역협상 타결? 결렬? 연장? 향후 가능한 세가지 시나리오',
  '10일 0시 관세 인상  美  최대 압박  속 협상 나서는 中'],
 ['불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저',
  '1분기 경상수지 흑자 112.5억달러 2012년 이후 최소',
  '경상수지 83개월 연속 흑자 끊기나 외국인 배당금이 좌우',
  '경상수지 83개월 흑자행진에도...규모는 7년 만에 최소',
  '수출 부진  1분기 경상흑자 112.5억달러...6년9개월만에  최소',
  '수출 부진에 경상수지  휘청  6년9개월만 흑자 최소(종합)',
  '1분기 경상수지 6년9개월 만에 최저 수출 부진 속  불황형 흑자',
  '수출 부진  쪼그라든 경상흑자 4월 적자 전환하나?'],
 ['엠플러스, 70억 2차전지 제조설비 공급.. 해외수주 증가',
  '엠플러스, SK이노베이션 헝가리 법인과 70억 수주 계약',
  '엠플러스, SK이노베이션 헝가리 법인과 70억원 규모 수주 계약',
  '엠플러스, SK이노베이션 헝가리법인과 70억원 공급계약',
  '엠플러스 수주공시 - 2차전지 조립공정 제조 설비 공급 계약 69.7억원 (매출액대비 8.93%)',
  '엠플러스, SK이노 헝가리 법인과 70억 규모 수주 계약'],
 ['하나은행-주금공, 한부모가족 전세대출 요건완화 금리인하',
  'KEB하나은행-주금공, 한부모가족에 전세대출 지원',
  '하나은행, 한부모 가족 전세자금 대출 실시',
  'KEB하나은행, 한부모 가족에 전세금 90% 대출',
  'KEB하

In [8]:
# Detail test
query_body = scouter.make_keyword_query_body('트럼프OPEC발언')
doc_info_list = scouter.search(query_body, data_type="detail_topic", max_num_doc=1000, trim_lower=False)

Query : 트럼프OPEC발언
Scroll idx : 1 (1 docs)
Total retrieved Doc # :  1



In [9]:
doc_info_list[0]['section'][1][1]

{'naverUrl': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=262&oid=003&aid=0009080155',
 'analyzed_text': {'sentence': [{'phrase_dependency': [{'weight': 0,
      'end': 1,
      'key_begin': 0,
      'id': 0,
      'label': 'NP_SBJ',
      'text': '트럼프 대통령은',
      'head_phrase': 2,
      'begin': 0,
      'sub_phrase': [],
      'element': []},
     {'weight': 0,
      'end': 3,
      'key_begin': 2,
      'id': 1,
      'label': 'NP_OBJ',
      'text': '이날 트위터를',
      'head_phrase': 2,
      'begin': 2,
      'sub_phrase': [],
      'element': []},
     {'weight': 0,
      'end': 4,
      'key_begin': 4,
      'id': 2,
      'label': 'S',
      'text': 'P#0@SBJ은 P#1@OBJ를 통해',
      'head_phrase': 5,
      'begin': 0,
      'sub_phrase': [0, 1],
      'element': []},
     {'weight': 0,
      'end': 5,
      'key_begin': 5,
      'id': 3,
      'label': 'NP_SBJ',
      'text': '유가가',
      'head_phrase': 5,
      'begin': 5,
      'sub_phrase': [],
      'elem

In [ ]:
# Detail test
query_body = scouter.make_keyword_query_body(''.join(clusters[1]['keywords']), filters=['news_id', 'postingDate', 'newsTitle', 'extContent', 'postingDate', 'analyzed_text'])
doc_info_list = scouter.search(query_body, data_type="detail_topic", max_num_doc=1000, trim_lower=False)

In [13]:
topic_doc

{'postingDate': '2019-03-28',
 'flag': True,
 'clusters': [{'keywords': ['터키', '리라', '급락'],
   'news_source': [{'naverUrl': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=262&oid=011&aid=0003529339',
     'extContentLength': 1408,
     'companyName': '서울경제',
     'newsTitle': '리라화 지키려다 증시도 폭락..에르도안 또 자충수',
     'news_id_2': '2019_8225624',
     'caption': [''],
     'postingDate': '2019-03-28',
     'originalUrl': 'https://www.sedaily.com/NewsView/1VGRLFN3WA',
     'category': '글로벌경제',
     'extContent': '27일(현지시간) 터키리라화를 해외에서 차입할 때 적용되는 오버나이트 스와프 금리가 무려 1,200%나 급등하자 외환시장에서는 괴성이 터져 나왔다. 지난주까지만 해도 20% 초반에 불과했던 금리가 지난 2001년 이후 최고치를 찍으며 차입 비용이 상상도 못할 수준까지 치솟았기 때문이다. 리라화 구하기가 하늘의 별 따기만큼 어려워지자 투자자들은 이날 리라화 대신 터키 주식과 채권을 부랴부랴 팔아치웠다. 지난해 신흥시장 급락 당시 진원지로 지목됐던 터키 금융시장이 최근 다시 요동치는 가운데 레제프 타이이프 에르도안 터키 대통령의 막무가내식 처방이 논란의 중심에 섰다. 에르도안 대통령이 이달 31일 지방선거를 앞두고 외환시장 안정을 목표로 무리한 유동성 차단정책을 밀어붙인 결과 역효과가 초래되고 있다는 것이다. 이미 총리 3선, 대통령 재선으로 무려 30년 장기집권의 길을 연 에르도안 대통령이 지방선거에서 확실한 승리를 거두고 

In [ ]:
query_body = ScouterHandler.make_count_query_body(keyword, from_date="2019-05-01", to_date="2019-12-31")
trend_data = scouter.search_for_trend(query_body, max_num_doc=100)

In [ ]:
from collections import Counter
Counter({d['key_as_string']:d['doc_count'] for d in trend_data['aggregations']['date_range']['buckets'][0]['group_by_date']['buckets']})

# 2. Multi-document Summary

In [7]:
%%time




CPU times: user 2.13 s, sys: 244 ms, total: 2.37 s
Wall time: 2.24 s


In [11]:
%%time
multi_doc_summarizer.process_data([doc['extContent'] for doc in clusters[0]['news_source']][:8])

CPU times: user 3.15 s, sys: 976 ms, total: 4.12 s
Wall time: 4.12 s


'1분기 경상수지 흑자 규모가 6년9개월래 최소폭 축소됐다. 전년동기 대비 9.4% 줄었다. 전년 동월비 9.4% 감소한 수치다. 1분기(4분기) 이후 가장 저조한 성적을 보였기 때문으로 풀이된다. 1분기 경제성장률은 전분기보다 9.7% 감소했다. 전년. . . . . . . . '

# 3. TimelineSummary

In [13]:
keyword = '미국 중국 무역'
outpath = './results/'

In [14]:
%%time
scouter = ScouterHandler(addr=default_addr, size=1000)
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'postingDate', 'extContent', 'extract'])
doc_info_list = scouter.search(query_body, data_type=default_timeline_idx, max_num_doc=200, trim_lower=False)

Query : 미국 중국 무역


[2019-11-21 07:19:06,660 INFO] GET http://155.230.34.145:9200/summary_by_time/_search?scroll=10m&size=200 [status:200 request:0.914s]


Scroll idx : 1 (200 docs)
Total retrieved Doc # :  200

CPU times: user 360 ms, sys: 40 ms, total: 400 ms
Wall time: 1.26 s


In [15]:
%%time
timeline_summ = TimelineSummary(keyword, outpath)
summary, _ = timeline_summ.process_data(doc_info_list)
dates = [sentence['date'] for sentence in summary]
timeline_sen = [' '.join(sentence['compResult']) for sentence in summary]
normal_sentence = [sentence['article'][0]['sentence'].strip() for sentence in summary]
score = [sentence['article'][2]['score'] for sentence in summary]

A :  0.05431962013244629
B :  0.18769121170043945
C :  0.009619712829589844
C-sub :  0.02051830291748047
 >> phase 4. calculate next nodes
timeline
['2017-09-20', '2018-11-15', '2018-11-29', '2019-01-08', '2019-05-07', '2019-05-11']
date_idx: 1/6
date_idx: 2/6
date_idx: 3/6
date_idx: 4/6
date_idx: 5/6
 >> phase 4. Make beamsearch file
D :  0.004110813140869141
E :  2.922905206680298
CPU times: user 444 ms, sys: 364 ms, total: 808 ms
Wall time: 3.2 s


In [ ]:
list(zip(dates, normal_sentence))

In [ ]:
list(zip(dates, timeline_sen))

# 4. Image Selection

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [4]:
%%time




weight_best loaded

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


graph defined
CPU times: user 5.18 s, sys: 356 ms, total: 5.54 s
Wall time: 5.51 s


In [7]:
%%time
imageSelector.process_data("트럼프의 뒤집기 전술에 딜레마 빠진 中 협상 해도 문제", "", download_limit=50)

TypeError: process_data() missing 1 required positional argument: 'image_save_path'

In [8]:
%%time
imageSelector.process_data("트럼프의 뒤집기 전술에 딜레마 빠진 中 협상 해도 문제", "./a.jpg", download_limit=50)

[2019-11-28 06:22:39,449 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.012s]


query already used before
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : others
15.jpg : , Predict : others
16.jpg : , Predict : others
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : others
23.jpg : , Predict : others
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : graph
33.jpg : , Predict : others
34.jpg : , Predict : others

In [12]:
%%time
imageSelector.process_data("당장 이란 군부는 이날 미국이 이란산 원유수출에 대한 제한적 제재유예를 연장하지 않은 것과 관련해 워너유 수송로인 호르무츠해협을 봉쇄하겠다고 경고하며 맞대응했다.", "./a.jpg", download_limit=50)

query is new
query pre-processed
try image download again
try image download again
try image download again
try image download again
query does not have any images on Google search engine
CPU times: user 3.61 s, sys: 56 ms, total: 3.66 s
Wall time: 5.37 s


False

In [13]:
%%time
imageSelector.process_data("당장 이란 군부는 이날 미국이 이란산 원유수출에 대한 제한적 제재유예를 연장하지 않은 것과 관련해 워너유 수송로인 호르무츠해협을 봉쇄하겠다고 경고하며 맞대응했다.", "./a.jpg", download_limit=50)

query is new
query pre-processed
try image download again
try image download again
try image download again
try image download again
query does not have any images on Google search engine
CPU times: user 3.63 s, sys: 60 ms, total: 3.69 s
Wall time: 5.37 s


False

In [14]:
try_iter = 0
while try_iter < 5:
    try_iter+= 1
    
try_iter

5

In [5]:
import re

date_list = ["2019-05-07", "2019-05-08", "2019-05-09", "2019-03-28"]
#for date in date_list:
#    os.mkdir("./results/"+date)

result_list = [
    [
        "트럼프의 뒤집기 전술에 딜레마 빠진 中 협상 해도 문제",
        "韓외환보유액 순위 7개월만에 9위로 밀려 4월말 12억달러",
        "엠플러스, 70억 2차전지 제조설비 공급.. 해외수주 증가",
        "하나은행-주금공, 한부모가족 전세대출 요건완화 금리인하",
        "관세전쟁 악몽 재현  코스피 2170선붕괴"
    ],
    [
        "불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저",
        "트럼프 추가 관세 위협 속 미중 9∼10일 무역협상 나선다",
        "EU, 유로존 GDP 성장률 하향 조정 1.3% 1.2%(종합)",
        "개인도 주택금융공사 발행 MBS 투자 가능해진다",
        "글로벌 기업 CFO 35.6%  브렉시트 시한 또 연장될 것"
    ],
    [
        "끝나지 않은 무역전쟁 악몽",
        "중고차 대출한도, 차값의 110% 이내로 제한된다",
        "비씨카드 이제 중국에서 모바일로 결제하세요",
        "한은, 새로운 금융상황지수 도입... 2017년 4분기 이후 완화기조 지속",
        "삼성자산운용, 개인연금 설정액 1조 돌파"
    ],
    [
        "선거 임박 터키 리라 방어 임시방편에 금융시장 요동",
        "전세 월세 내준 임대가구 금융부채 372조원",
        "조양호 대표이사 박탈 영향은 미미할 것",
        "트럼프, OPEC에 유가 상승 경고  OPEC 진정해야",
        "상장법인 재감사 비율 증가세 보수는 2.6배 높아"
    ]
]
for date, sen_list in zip(date_list, result_list):
    print("[*] DATE : " + date)
    for i, sen in enumerate(sen_list):
        sen = re.sub("\s+", " ", sen)
        print(i, sen)
        imageSelector.process_data(sen, "./results/{}/daily_{}.jpg".format(date, i))

[*] DATE : 2019-05-07
0 트럼프의 뒤집기 전술에 딜레마 빠진 中 협상 해도 문제
10.515323400497437
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : graph
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predi

0.19605398178100586
recommended image : 30.jpg
3.9715960025787354
[*] DATE : 2019-05-08
0 불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저
11.173689603805542
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , P

vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : others
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : others
12.jpg : , Predict : graph
13.jpg : , Predict : others
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : others
36.jpg : , Predict : othe

0.1919093132019043
recommended image : 24.jpg
4.003591299057007
4 삼성자산운용, 개인연금 설정액 1조 돌파
10.96693205833435
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : graph
2.jpg : , Predict : others
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict

vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : others
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : graph
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : others
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : others
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : graph
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : grap

In [10]:
result_list = [
    [
        "지난해(181억7000만달러)에 비해 증가 폭이 줄었지만 한국 외환보유액은 10년간 증가세를 보이며 두 배로 확대됐다.",
        "지난달 외환보유액이 4,055억 달러로 사상 최대 기록을 다시 갈아치웠다.",
        "외환보유액 1위 중국 3만879억달러, 2위는 일본 1만2793억달러 우리나라의 외환보유액 규모는 2월말 기준 소폭 감소하며 세계 8위 수준(1월)을 유지한 것으로 나타났다.",
        "우리나라의 외환보유액은 지난해 8월말 4011억달러을 기록해 국가별 순위에서 인도(4001억달러)를 제치고 9위에서 8위로 올라선 바 있다."
    ],
    [
        "15일(현지시간) 영국 하원에서 브렉시트 합의안이 큰 표 차로 부결되며 불확실성이 커진 점이 원/달러 환율 상승 배경인 것으로 풀이된다.",
        "테리사 메이 영국 총리는 표결 직후 성명을 통해 예고한 대로 13일 노딜 브렉시트 여부를 하원 표결에 부치겠다고 밝혔다.",
        "21~22일 EU정상회담 메이 빈손회담 가능성 커져 유럽연합(EU)이 브렉시트(영국의 EU 탈퇴) 시기 연기해달라는 영국의 요청에 영국 의회가 합의문을 승인해야 한다고 밝혔다."
    ],
    [
        "도널드 트럼프 미국 대통령이 오는 3월 1일까지였던 중국과의 무역 협상 시한을 연장하기로 했다.",
        "겅솽 대변인의 발표를 앞두고 중국 상무부도 류허 중국 부총리가 오는 9~10일 예정대로 미국에서 협상을 벌일 것이라고 성명을 내기도 했다.",
        "지난해 8월 22~23일 워싱턴DC에서는 왕서우원 중국 상무부 부부장과 데이비드 멀패스 미국 재무부 차관이 이끈 양국 협상 대표단이 무역협상을 진행했다.",
        "중국 정부는 미국의 관세 폭탄 에 대응해 6월 1일부터 600억 달러 규모의 미국산 제품에 대해 5∼25%의 보복 관세를 부과하기로 했다.",
        "미국 정부는 23일(현지시간) 자국 통화를 절하하는 국가들에 상계관세를 부과하는 규정을 추진한다고 밝혔다.",
        "미국 재무부는 28일(현지시간) 한국과 중국, 일본을 포함한 9개국을 환율 관찰대상국으로 지정, 발표했다."
    ],
    [
        "하지만 14개 산유국으로 구성된 OPEC과 러시아 등이 가격 정상화를 위해 올해부터 하루 120만 배럴에 해당하는 대규모 감산을 결정했다.",
        "서부텍사스산원유(WTI)는 2일 전 거래일 대비 2.25% 오른 배럴당 46.54달러에 장을 마쳤으며 북해산 브렌트유는 같은날 장중 5.1% 가까이 뛰기도 했다.",
        "25일(현지시간) 트럼프 대통령은 자신의 트위터 계정을 통해 유가가 너무 높다며 OPEC은 제발 진정하라고 촉구했다.",
        "전날 OPEC(석유수출국기구)의 감산을 비판한 도널드 트럼프 미국 대통령의 트윗에도 불구하고 OPEC이 감산을 계속하겠다고 밝히면서다.",
        "당장 이란 군부는 이날 미국이 이란산 원유수출에 대한 제한적 제재유예를 연장하지 않은 것과 관련해 워너유 수송로인 호르무츠해협을 봉쇄하겠다고 경고하며 맞대응했다.",
        "미국 내 가솔린 소비자가는 이날 현재 갤런당 2.84달러에 달해 미국 소비자들의 심리적 저항선인 3달러에 근접하고 있다."
    ]
]

for date, sen_list in zip(date_list[-1:], result_list[-1:]):
    print("[*] DATE : " + date)
    for i, sen in enumerate(sen_list[-1:], start=len(sen_list)-1):
        sen = re.sub("\s+", " ", sen)
        print(i, sen)
        imageSelector.process_data(sen, "./results/{}/timeline_{}.jpg".format(date, i))

[*] DATE : 2019-03-28
5 미국 내 가솔린 소비자가는 이날 현재 갤런당 2.84달러에 달해 미국 소비자들의 심리적 저항선인 3달러에 근접하고 있다.
3.62126088142395
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
(9, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
0.05882668495178223
recommended image : 5.jpg
3.847708225250244


# 5. Outlook

In [22]:
%%time



keyword = '삼성전자'
outpath = 'result/'
today = '2017-06-11'


sentences = forecast.process_data(doc_info_list, today)
sentences[0:10]

/workspace/wise_reporter/modules/forecast/prediction.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = a(output)


CPU times: user 1.42 s, sys: 8 ms, total: 1.43 s
Wall time: 791 ms


[' 삼성전자의 올 2분기 실적이 매출액 58조9000억원, 영업이익 14조500억원으로 전년 동기에 비해 각각 15.7%, 72.5% 증가할 것으로 전망, 특히 영업이익이 시장의 기대치 12조9000억원을 상회할 것이라고 내다봤다.',
 ' 이베스트투자증권 어규진 연구원은  2분기에 메모리 가격 상승 및 OLED 패널 판매 호조가 지속되며 반도체와 디스플레이의 부품 사업부 실적 호조세가 지속될 전망 이라며  휴대폰 부분은 갤럭시S8 판매 호조에 따른 평균 판매가 상승 효과로 전작 대비 늦은 출시에도 견조한 실적을 유지할 전망 이라고 말했다.',
 ' 흥국증권 이민희 연구원은  삼성전자 2분기 영업이익 전망을 12조1000억원에서 13조원으로 상향 조정한다 며  메모리 가격 전망치 상향 조정과  노트7  리퍼폰 판매량을 일부 반영했다 고 설명했다.']

In [20]:
doc_info_list[0].keys()

dict_keys(['extract', 'extContent', 'rel_score', 'id', 'writetime'])